In [29]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [30]:
data = []
N =443
with open(r'data.txt', 'r') as f:
    file_lines = f.readlines()
for line in file_lines:
    line =line.replace("\n", "")
    line = line.split(' ')
    for i in line:
        try:
            data.append(int(i))
        except:
            continue
distance_data= np.array(data).reshape((N,N))

In [31]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_data
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

## Routing Model

In [32]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)
routing = pywrapcp.RoutingModel(manager)

In [33]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  

## Set cost of travel

In [34]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

## Set search Parameters

In [35]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

In [36]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"

In [37]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 2763 miles
Route for vehicle 0:
 0 -> 269 -> 166 -> 87 -> 427 -> 423 -> 422 -> 442 -> 392 -> 388 -> 441 -> 437 -> 436 -> 434 -> 414 -> 402 -> 433 -> 428 -> 426 -> 432 -> 409 -> 404 -> 311 -> 415 -> 431 -> 413 -> 395 -> 430 -> 418 -> 412 -> 429 -> 425 -> 424 -> 420 -> 111 -> 194 -> 24 -> 301 -> 242 -> 231 -> 155 -> 108 -> 165 -> 419 -> 277 -> 196 -> 101 -> 224 -> 227 -> 331 -> 195 -> 176 -> 69 -> 190 -> 104 -> 46 -> 284 -> 247 -> 233 -> 173 -> 107 -> 114 -> 184 -> 254 -> 93 -> 105 -> 323 -> 362 -> 379 -> 361 -> 278 -> 271 -> 360 -> 106 -> 342 -> 283 -> 302 -> 356 -> 186 -> 171 -> 160 -> 84 -> 55 -> 349 -> 332 -> 213 -> 294 -> 115 -> 210 -> 91 -> 193 -> 45 -> 28 -> 6 -> 65 -> 256 -> 159 -> 335 -> 312 -> 371 -> 347 -> 264 -> 255 -> 346 -> 318 -> 316 -> 174 -> 267 -> 344 -> 41 -> 345 -> 62 -> 188 -> 74 -> 26 -> 187 -> 83 -> 51 -> 343 -> 244 -> 232 -> 36 -> 372 -> 52 -> 341 -> 38 -> 370 -> 200 -> 49 -> 151 -> 85 -> 306 -> 340 -> 211 -> 48 -> 272 -> 339 -> 265 -> 25 -> 336 -> 229 

In [38]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

In [39]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
  print('Route', i, route)

Route 0 [0, 269, 166, 87, 427, 423, 422, 442, 392, 388, 441, 437, 436, 434, 414, 402, 433, 428, 426, 432, 409, 404, 311, 415, 431, 413, 395, 430, 418, 412, 429, 425, 424, 420, 111, 194, 24, 301, 242, 231, 155, 108, 165, 419, 277, 196, 101, 224, 227, 331, 195, 176, 69, 190, 104, 46, 284, 247, 233, 173, 107, 114, 184, 254, 93, 105, 323, 362, 379, 361, 278, 271, 360, 106, 342, 283, 302, 356, 186, 171, 160, 84, 55, 349, 332, 213, 294, 115, 210, 91, 193, 45, 28, 6, 65, 256, 159, 335, 312, 371, 347, 264, 255, 346, 318, 316, 174, 267, 344, 41, 345, 62, 188, 74, 26, 187, 83, 51, 343, 244, 232, 36, 372, 52, 341, 38, 370, 200, 49, 151, 85, 306, 340, 211, 48, 272, 339, 265, 25, 336, 229, 179, 126, 125, 438, 359, 94, 39, 334, 243, 68, 50, 22, 330, 257, 5, 177, 375, 304, 329, 215, 221, 88, 303, 279, 238, 327, 207, 162, 421, 130, 134, 80, 181, 172, 234, 137, 78, 47, 143, 8, 64, 322, 117, 164, 124, 122, 309, 261, 297, 183, 307, 20, 305, 248, 236, 63, 35, 170, 112, 110, 296, 169, 30, 33, 58, 295, 57, 

In [40]:
np.array(routes[0])

array([  0, 269, 166,  87, 427, 423, 422, 442, 392, 388, 441, 437, 436,
       434, 414, 402, 433, 428, 426, 432, 409, 404, 311, 415, 431, 413,
       395, 430, 418, 412, 429, 425, 424, 420, 111, 194,  24, 301, 242,
       231, 155, 108, 165, 419, 277, 196, 101, 224, 227, 331, 195, 176,
        69, 190, 104,  46, 284, 247, 233, 173, 107, 114, 184, 254,  93,
       105, 323, 362, 379, 361, 278, 271, 360, 106, 342, 283, 302, 356,
       186, 171, 160,  84,  55, 349, 332, 213, 294, 115, 210,  91, 193,
        45,  28,   6,  65, 256, 159, 335, 312, 371, 347, 264, 255, 346,
       318, 316, 174, 267, 344,  41, 345,  62, 188,  74,  26, 187,  83,
        51, 343, 244, 232,  36, 372,  52, 341,  38, 370, 200,  49, 151,
        85, 306, 340, 211,  48, 272, 339, 265,  25, 336, 229, 179, 126,
       125, 438, 359,  94,  39, 334, 243,  68,  50,  22, 330, 257,   5,
       177, 375, 304, 329, 215, 221,  88, 303, 279, 238, 327, 207, 162,
       421, 130, 134,  80, 181, 172, 234, 137,  78,  47, 143,   